In [2]:
import requests
import time
import execjs
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
from multiprocessing import Process

In [4]:
def getAllCode():
    url = 'http://fund.eastmoney.com/js/fundcode_search.js'
    content = requests.get(url)
    jsContent = execjs.compile(content.text)
    rawData = jsContent.eval('r')
    allCode = []
    for code in rawData:
        allCode.append(code[0])
    return allCode


In [138]:
# fund_narrow_pd=pd.read_csv('./narrow/fund_narrow_pd.csv',low_memory=False, header = None)
# worth_pd文件 最长是 202001
# rateInSimilar_pd最长是040001
# grandTotal_pd 最长是 005503
grandTotal_pd=pd.read_csv('./narrow/grandTotal_pd.csv',low_memory=False, header = None)
rateInSimilar_pd=pd.read_csv('./narrow/rateInSimilar_pd.csv',low_memory=False, header = None)
worth_pd = pd.read_csv(open('./1-8926/san/worth_pd.csv','r'),low_memory=False, header = None)
fund_wide_pd_type=pd.read_csv('./wide/fund_wide_pd.csv',low_memory=False)

In [144]:
# df.iloc = df['col2'].astype('float64')
worth_pd = worth_pd.fillna(0)
grandTotal_pd = grandTotal_pd.fillna(0)
rateInSimilar_pd = rateInSimilar_pd.fillna(0)

print(grandTotal_pd.info())
print(rateInSimilar_pd.info())
print(worth_pd.info())
print(fund_wide_pd_type.info())
# 拆分：每份基金份额分拆
# 拆分：每份基金份额折算
# 分红：每份派现金
# worth_pd.loc[:,2:].astype('float')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29316 entries, 0 to 29315
Columns: 136 entries, 0 to 135
dtypes: float64(134), int64(1), object(1)
memory usage: 30.4+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30452 entries, 0 to 30451
Columns: 4303 entries, 0 to 4302
dtypes: float64(4301), int64(1), object(1)
memory usage: 999.7+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38075 entries, 0 to 38074
Columns: 4345 entries, 0 to 4344
dtypes: float64(4343), int64(1), object(1)
memory usage: 1.2+ GB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7623 entries, 0 to 7622
Data columns (total 32 columns):
name           7623 non-null object
code           7623 non-null object
类型             7623 non-null object
原费率            5857 non-null object
现费率            5851 non-null object
最小申购金额         5853 non-null object
近一年收益率         5911 non-null object
近6月收益率         6575 non-null object
近三月收益率         6991 non-null object
近一月收益率         7231 non-null obj

In [70]:
# 找 N 年以上的基金
def get_year_funds(years, allCode):
    funds_year = []
#     allCode = ['000008']
    for code in allCode:
#         print(code)
        fund_day = rateInSimilar_pd.loc[(rateInSimilar_pd[0] == "时间戳") & (rateInSimilar_pd[1] ==  int(code))]
        if fund_day.shape[0] > 0:
            fund_day = fund_day.T
#             print(fund_day.shape)
#             print(fund_day)
            day_c = fund_day[fund_day.iloc[:,0] != 0].shape[0]
            if day_c >= 365 * years + 10:
#                 print("code",code,"day_c",day_c)
                funds_year.append(code)
    return funds_year

In [71]:
allCode = getAllCode()
funds_5y = get_year_funds(5,allCode)
pd.DataFrame(funds_5y).to_csv("./funds_5y.csv", header=False, index=False)

In [72]:
funds_3y = get_year_funds(3,allCode)
pd.DataFrame(funds_3y).to_csv("./funds_3y.csv", header=False, index=False)

In [74]:
print(len(funds_3y))

1705


In [128]:
# 近 N 年 同类绩效均值 n(0-1)
def get_similar_mean(year):
    if year <= 3:
        file_year = 3
    if year >3 and year <= 5:
        file_year = 5
    fund_similar_mean_list = []
    fund_similar_code_list = []
    fund_year_pd = pd.read_csv(
        './funds_' + str(file_year) + 'y.csv', low_memory=False, header=None)
    fund_year_list = fund_year_pd.values.tolist()
    
#     fund_year_list = [1,11]
    for code in fund_year_list:
#         start1 = time.time()
        code = code[0]
#         print(code)
        fund_similar = rateInSimilar_pd.loc[(rateInSimilar_pd[0] == "同类排名百分比")
                                          & (rateInSimilar_pd[1] == code)]
#         fund_similar.drop([0, 1, 2], axis=1, inplace=True) # 删除前三列无用数据
        fund_similar = fund_similar.T
        fund_similar = fund_similar.iloc[2:,:]
        fund_similar = fund_similar[fund_similar.iloc[:, 0] > 0] # 只要有效数据
        if fund_similar.shape[0] >= year * 365:
            fund_similar = fund_similar.iloc[0: round(year*365) , :]  # 找 x 天内的数据,取前 n 行
#             print(fund_similar.shape)
            mean = fund_similar.mean(0).iloc[0] # 求平均
            fund_similar_code_list.append(code)
            fund_similar_mean_list.append(mean)
            fund_similar_mean_dict = {
                "code" : fund_similar_code_list,
               "同类平均值" : fund_similar_mean_list
            } 
#         end1 = time.time()
#         print("for 用时{}".format((end1-start1)))
    fund_similar_mean_pd =  pd.DataFrame(fund_similar_mean_dict)
    
    return fund_similar_mean_pd

In [87]:
def get_similar_top(similar_mean,top):
#     排序
    similar_mean.sort_values(by = '同类平均值', inplace=True, ascending=False)
#     获取长度
    pd_len = similar_mean.shape[0]
#     取前三分之一
    similar_top = similar_mean.head(round(pd_len * top))
    return similar_top

In [88]:
# 统计5 年来前 30%的基金
for i in range(1,6,1):
    fund_similar_mean_pd = get_similar_mean(i)
    fund_similar_top_pd = get_similar_top(fund_similar_mean_pd, 0.3)
    fund_similar_top_pd.to_csv("./fund_similar_top" + str(i) +"y_pd.csv", index=False)
# print(fund_similar_mean_pd)

In [129]:
fund_similar_mean_pd = get_similar_mean(0.5)
fund_similar_top_pd = get_similar_top(fund_similar_mean_pd, 0.3)
fund_similar_top_pd.to_csv("./fund_similar_top" + str(0.5) +"y_pd.csv", index=False)

In [136]:
fund_similar_top_6m_pd = pd.read_csv('./fund_similar_top0.5y_pd.csv',low_memory=False, header=None)
fund_similar_top_1y_pd = pd.read_csv('./fund_similar_top1y_pd.csv',low_memory=False, header=None)
fund_similar_top_2y_pd = pd.read_csv('./fund_similar_top2y_pd.csv',low_memory=False, header=None)
fund_similar_top_3y_pd = pd.read_csv('./fund_similar_top3y_pd.csv',low_memory=False, header=None)
fund_similar_top_4y_pd = pd.read_csv('./fund_similar_top4y_pd.csv',low_memory=False, header=None)
fund_similar_top_5y_pd = pd.read_csv('./fund_similar_top5y_pd.csv',low_memory=False, header=None)
# print(fund_similar_top_1y_pd.shape)
# print(fund_similar_top_2y_pd.shape)
m1 = pd.merge(fund_similar_top_6m_pd,fund_similar_top_1y_pd,how='inner',on = 0)
m2 = pd.merge(m1,fund_similar_top_2y_pd,how='inner',on = 0)
m3 = pd.merge(m2,fund_similar_top_3y_pd,how='inner',on = 0)
m4 = pd.merge(m3,fund_similar_top_4y_pd,how='inner',on = 0)
m5 = pd.merge(m4,fund_similar_top_5y_pd,how='inner',on = 0)
m5.to_csv("./fund_similar_top_6m-5y_pd.csv", encoding='utf_8_sig')
print(m5.shape)

(57, 7)


In [285]:
# 获取 code 对应的类型
def get_fund_type_name(code):
    try:
        type_name = fund_wide_pd_type.loc[fund_wide_pd_type['code'] == code]['类型']
    #     print(len(type_name))
        if(len(type_name) > 0):
            return type_name.values[0]
        else:
            return ""
    except:
        return "111"

In [282]:
get_fund_type_name("1".)

000001


'混合型'

In [279]:
a = "1".zfill(6)
print(a)

000001


In [286]:
# 读取相同类型 top 30% 的文件
fund_similar_top_6m_5y_pd = pd.read_csv("./fund_similar_top_6m-5y_pd.csv", encoding='utf_8_sig')
# 创建空列，dtype = object 可以存多个字符
type_name_np = np.empty(fund_similar_top_6m_5y_pd.shape[0], dtype = object)
# 遍历 df
for i in range(0, len(fund_similar_top_6m_5y_pd)):
# for i in range(0, 3):
    code = fund_similar_top_6m_5y_pd.iloc[i]['0'] # 获取 code
#     print(code)
    type_name = get_fund_type_name(code.zfill(6)) # 获取 类型
    type_name_np[i] = type_name # 保存名称
# 插入type_name列
fund_similar_top_6m_5y_pd.insert(1,'type_name', type_name_np)
# 保存文件
fund_similar_top_6m_5y_pd.to_csv("./fund_similar_top_6m-5y_pd11.csv", encoding='utf_8_sig')
#     fund_similar_top_6m_5y_pd.iloc[i]['type_name'] = "a"
    
    
#     print(type_name)
#     print()
#     print(type(fund_similar_top_6m_5y_pd.iloc[i]['0']))   

In [239]:
# fund_similar_top_6m_5y_pd.iloc[3]['type_name'] = "a"
# fund_similar_top_6m_5y_pd.insert(3,'type_name', type_name)
# print(fund_similar_top_6m_5y_pd['type_name'])
print(type_name_np)

['类' '定' '债' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' ''
 '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' ''
 '' '' '' '' '' '' '' '' '' '']


In [ ]:
# 测试代码分割线

In [338]:
# mean = fund_similar_mean_pd.iloc[0,:].mean(0)
# print(fund_similar_mean_pd)
fund_similar_mean_pd_t = fund_similar_mean_pd
fund_similar_mean_pd_t.sort_values(by = '同类平均值', inplace=True, ascending=False)
pd_len = fund_similar_mean_pd_t.shape[0]
print(pd_len * 0.3)

print(fund_similar_mean_pd_t.shape)
# fund_similar_mean_pd_t = fund_similar_mean_pd_t.iloc[0:round(pd_len * 0.3)]
fund_similar_mean_pd_t = fund_similar_mean_pd_t.head(round(pd_len * 0.3))
print(fund_similar_mean_pd_t.shape)
# round(pd_len * 0.3)
# .head(3)


251.7
(839, 2)
(252, 2)


In [119]:
fund_similar = rateInSimilar_pd.loc[(rateInSimilar_pd[0] == "同类排名百分比")
                                  & (rateInSimilar_pd[1] == 100022)]
# fund_similar.drop([0, 1, 2], axis=1, inplace=True)  # 删除前三列无用数据
fund_similar = fund_similar.T
# fund_similar = fund_similar.iloc[2:,:]
print(fund_similar)
fund_similar = fund_similar[fund_similar.iloc[:, 0] > 0]  # 只要有效数据
#         还需要找 x 天内的数据
# print(fund_similar)
# print(fund_similar.iloc[:,5*365].shape)
# fund_similar = fund_similar.iloc[5*365,:]
print(fund_similar.iloc[0:5*365,:].shape)
mean = fund_similar.mean(0).iloc[0]  # 求平均
print(mean.shape)
print(mean)
print(type(mean))

        22711
0     同类排名百分比
1      100022
2        74.5
3       75.32
4          79
5       79.68
6       81.57
7        81.6
8       82.19
9       82.47
10      80.34
11       81.1
12      80.62
13      79.99
14      82.29
15      83.11
16      74.74
17      73.45
18      71.62
19       70.6
20      66.37
21      63.83
22      72.37
23      69.59
24      66.45
25      71.47
26      61.11
27      57.23
28      66.52
29      33.52
...       ...
4273        0
4274        0
4275        0
4276        0
4277        0
4278        0
4279        0
4280        0
4281        0
4282        0
4283        0
4284        0
4285        0
4286        0
4287        0
4288        0
4289        0
4290        0
4291        0
4292        0
4293        0
4294        0
4295        0
4296        0
4297        0
4298        0
4299        0
4300        0
4301        0
4302        0

[4303 rows x 1 columns]
(1825, 1)


IndexError: single positional indexer is out-of-bounds

In [239]:

fund_day = fund_narrow_pd.loc[(fund_narrow_pd[0] == "时间戳") & (fund_narrow_pd[1] ==  int("040023"))]
fund_day = fund_day.T
print("fund_day", fund_day.shape)

x1 = fund_day[fund_day.iloc[:,0] != 0]
print(x1.shape)
# print(x1)




fund_day (4344, 1)
(2002, 1)


Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, ...]
Index: []

[0 rows x 4344 columns]
